In [ ]:
#!pip3 install scapy

In [1]:
import requests
import pandas as pd

url = 'https://us-proxy.org/'

In [53]:
def fetch_list(url, port=8080, https='no', head_count=20):
    r = requests.get(url)
    html = r.content
    df_list = pd.read_html(html, parse_dates=True)
    ips = df_list[0].dropna()
    ips = ips.drop(columns=['Google'])
    print(ips)
    
    ips = ips.rename(columns={'Last Checked': 'last_checked', 'IP Address': 'ip'})
    new_cols = [e.lower() for e in list(ips.columns)]
    old_cols = list(ips.columns)
    columns = dict(zip(old_cols, new_cols))
    ips = ips.rename(columns=columns)  
    ips['port'] = ips['port'].astype(int)
    
    http_only = ips['https'] == 'no'
    ips = ips[http_only].drop(columns=['https'])
    http_only_port = ips['port'] == port
    ips = ips[http_only_port].drop(columns=['port'])
    
    ips = ips[['ip']].head(head_count)
    return ips.reset_index().drop(columns=['index'])

In [56]:
proxy_list = fetch_list(url, https='yes')
proxy_list

          IP Address    Port Code        Country    Anonymity Https  \
0     75.109.249.111  1111.0   US  United States  elite proxy    no   
1     139.229.20.220    80.0   US  United States  elite proxy    no   
2      67.205.151.68  8080.0   US  United States    anonymous    no   
3    104.207.134.206  8080.0   US  United States    anonymous    no   
4     68.183.115.230  8080.0   US  United States    anonymous    no   
..               ...     ...  ...            ...          ...   ...   
195   85.209.150.222  8085.0   US  United States  transparent    no   
196    85.208.209.67  8085.0   US  United States  transparent    no   
197   85.209.150.250  8085.0   US  United States  transparent    no   
198    85.208.209.74  8085.0   US  United States  transparent    no   
199    213.166.78.35  8085.0   US  United States  transparent    no   

             Last Checked  
0            1 minute ago  
1            1 minute ago  
2           2 minutes ago  
3           2 minutes ago  
4      

,ip
0,67.205.151.68
1,104.207.134.206
2,68.183.115.230
3,149.28.235.152
4,102.129.249.120
5,54.176.154.92
6,198.199.86.11
7,157.245.221.254
8,138.68.240.218
9,13.85.25.59


In [ ]:
import imaplib

IMAP_SERVER='imap.gmail.com'
IMAP_PORT=993

try:
 M = imaplib.IMAP4_SSL(IMAP_SERVER, IMAP_PORT)
 rc, resp = M.login('email address', 'password')
 print rc, resp
 M.logout()
except Exception, e:
 print e

In [46]:
from scapy.all import IP, ICMP, sr1

def ping(ip):
    icmp = IP(dst=ip)/ICMP()
    #IP defines the protocol for IP addresses
    #dst is the destination IP address
    #TCP defines the protocol for the ports
    resp = sr1(icmp, timeout=10)
    is_alive = resp is not None
    return is_alive

In [47]:
def get_active_proxies(proxy_list):
    active_proxies = []
    for idx, row in proxy_list.iterrows():
        if ping(row['ip']):
            active_proxies.append(row)
    return active_proxies

In [57]:
active_proxies = get_active_proxies(proxy_list)
active_proxies

OSError: [Errno 97] Address family not supported by protocol